<a href="https://colab.research.google.com/github/Santosh-Gupta/ScientificSummarizationDataSets/blob/master/PublicSectionalSummariesBertExtractive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Make sure you import/copy the sectional summary dataset, the folder name should be 'SemanticScholarAbstractSectionSummaryDataSet' and the path should be 'gdrive/My Drive/SemanticScholarAbstractSectionSummaryDataSet/'

Highly recommended that you switch the output for the final processed files to a folder in your google drive folder, so you can skip that step the next time.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import requests
import os
import random
import tarfile
import pandas as pd
import zipfile
import multiprocess
import json
from glob import glob

!apt-get install libmagic-dev
!pip install python-magic
import magic
mime = magic.Magic(mime=True)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1
Suggested packages:
  file
The following NEW packages will be installed:
  libmagic-dev libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 4 not upgraded.
Need to get 332 kB of archives.
After this operation, 5,549 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.2 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.2 [68.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-dev amd64 1:5.32-2ubuntu0.2 [79.5 kB]
Fetched 332 kB in 1s (435 kB/s)
Selecting previously unselected package libmagic-mg

In [0]:
!pip install torch
!pip install pytorch-pretrained-bert
!pip install tensorboardX
!pip install pyrouge
!pip install fastparquet

     |████████████████████████████████| 133kB 3.4MB/s 
     |████████████████████████████████| 655kB 46.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
Successfully built regex
     |████████████████████████████████| 225kB 3.4MB/s 
     |████████████████████████████████| 61kB 3.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/75/d3/0c/e5b04e15b6b87c42e980de3931d2686e14d36e045058983599
Successfully built pyrouge
     |████████████████████████████████| 153kB 3.5MB/s 
     |████████████████████████████████| 61kB 26.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/f3/27/fb/839c776ec8689ff9ee52ad3e91d7d4d848ac6d7545a127d5b0
  Stored in directory: /root/.cache/pip/wheels/be/36/81/0f93ba89a1cb7887c91937948519840a72c0ffdd57cac0ae8f
Successfully built fastparquet thrift


In [0]:
#install Java for stanford tokenizer

!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!java -version

openjdk version "11.0.3" 2019-04-16
OpenJDK Runtime Environment (build 11.0.3+7-Ubuntu-1ubuntu218.04.1)
OpenJDK 64-Bit Server VM (build 11.0.3+7-Ubuntu-1ubuntu218.04.1, mixed mode, sharing)


In [0]:
#download stanford tokenizer

url = 'http://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip'
r = requests.get(url)
with open('stanford-corenlp-full-2018-10-05.zip', 'wb') as f:
    f.write(r.content)

In [0]:
with zipfile.ZipFile('stanford-corenlp-full-2018-10-05.zip', 'r') as zip_ref:
    zip_ref.extractall('')

In [0]:
#check jar file

!jar -tvf stanford-corenlp-full-2018-10-05/stanford-corenlp-3.9.2.jar

#Note, the repo used is for this version of the stanford parser. If changing the version, you need to change this line
# in the repo
# https://github.com/Santosh-Gupta/BertSum/blob/master/src/prepro/data_builder.py#L235

     0 Mon Oct 08 23:22:50 UTC 2018 META-INF/
   176 Mon Oct 08 23:22:50 UTC 2018 META-INF/MANIFEST.MF
 54588 Mon Oct 08 22:56:12 UTC 2018 edu/stanford/nlp/pipeline/StanfordCoreNLP.class
 23392 Mon Oct 08 22:56:26 UTC 2018 edu/stanford/nlp/pipeline/CustomAnnotationSerializer.class
  7847 Mon Oct 08 22:56:28 UTC 2018 edu/stanford/nlp/process/PTBEscapingProcessor.class
  4767 Mon Oct 08 22:56:14 UTC 2018 edu/stanford/nlp/dcoref/Constants.class
   688 Mon Oct 08 22:56:16 UTC 2018 edu/stanford/nlp/dcoref/sievepasses/AliasMatch.class
   760 Mon Oct 08 22:56:16 UTC 2018 edu/stanford/nlp/dcoref/sievepasses/CorefDictionaryMatch.class
 22345 Mon Oct 08 22:56:16 UTC 2018 edu/stanford/nlp/dcoref/sievepasses/DeterministicCorefSieve.class
   534 Mon Oct 08 22:56:16 UTC 2018 edu/stanford/nlp/dcoref/sievepasses/DiscourseMatch.class
   542 Mon Oct 08 22:56:16 UTC 2018 edu/stanford/nlp/dcoref/sievepasses/ExactStringMatch.class
   754 Mon Oct 08 22:56:16 UTC 2018 edu/stanford/nlp/dcoref/sievepasses/Lexi

In [0]:
#Test Stanford Tokenizer 

!echo "Please tokenize this text." | java -cp stanford-corenlp-full-2018-10-05/stanford-corenlp-3.9.2.jar edu.stanford.nlp.process.PTBTokenizer

Please
tokenize
this
text
.
PTBTokenizer tokenized 5 tokens at 39.16 tokens per second.


In [0]:
#Get forked Bert summarization repos

!git clone https://github.com/Santosh-Gupta/BertSum.git

Cloning into 'BertSum'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 345 (delta 36), reused 0 (delta 0), pack-reused 295
Receiving objects: 100% (345/345), 15.04 MiB | 17.75 MiB/s, done.
Resolving deltas: 100% (204/204), done.


In [0]:
if not os.path.exists('Tokenized'):
    os.makedirs('Tokenized')

In [0]:
if not os.path.exists('testDir'):
    os.makedirs('testDir')

In [0]:
filelist = glob('gdrive/My Drive/SemanticScholarAbstractSectionSummaryDataSet/*')

In [0]:
# for file in filelist:
#     print(file)
#     df = pd.read_parquet(file, engine='fastparquet')
#     for index, row in df.iterrows():
#         sampletext =  row['paperSection'] + '@highlight' + row['Summary']
#         text_file = open('testDir/testtt'+str(index)+'.story', "w")
#         text_file.write(sampletext)
#         text_file.close()

In [0]:
if not os.path.exists('testDir2'):
    os.makedirs('testDir2')

Process files from gdrive

In [0]:
import threading


In [0]:
def thread_function(pfile):
    df = pd.read_parquet(pfile, engine='fastparquet')
    for index, row in df.iterrows():
        sampletext =  row['paperSection'] + '@highlight' + row['Summary']
        text_file = open('testDir2/placeholder'+pfile[82:86] + 'z' + str(index)+'.story', "w")
        text_file.write(sampletext)
        text_file.close()

In [0]:
a_threads = []
j = 0
for file in filelist:
    print(file)
    x = threading.Thread(target=thread_function, args=(file,))
    x.start()
    a_threads.append(x)
    if j%5 == 0:
        for thread in a_threads:
            thread.join()
    j=j+1
    
for thread in a_threads:
    thread.join()
    

gdrive/My Drive/SemanticScholarAbstractSectionSummaryDataSet/AbstractSummaryDataSet01.parquet.gzip


Tokenize processed files

In [0]:
%%capture
!python BertSum/src/preprocess.py -mode tokenize -raw_path 'testDir2' -save_path 'Tokenized' -log_file 'cnndm2.log'

In [0]:
len(os.listdir('Tokenized'))

49224

In [0]:
len(os.listdir('testDir2'))

49224

In [0]:
len('testDir2')

8

In [0]:
if not os.path.exists('NextDir'):
    os.makedirs('NextDir')

In [0]:
if not os.path.exists('FakeMapPath'):
    os.makedirs('FakeMapPath')

Process tokenized files

In [0]:
# %%capture
!python BertSum/src/preprocess.py -mode format_to_lines -raw_path 'Tokenized' -save_path 'NextDir/Files'  -log_file 'cnndm2.log' -map_path 'FakeMapPath' -lower

In [0]:
os.listdir('Tokenized')[0:10]

['placeholdert01.z47029.story.json',
 'placeholdert01.z4406.story.json',
 'placeholdert01.z26332.story.json',
 'placeholdert01.z373.story.json',
 'placeholdert01.z45073.story.json',
 'placeholdert01.z28714.story.json',
 'placeholdert01.z15857.story.json',
 'placeholdert01.z4173.story.json',
 'placeholdert01.z24630.story.json',
 'placeholdert01.z17031.story.json']

In [0]:
if not os.path.exists('FinalBertData'):
    os.makedirs('FinalBertData')

In [0]:

#!python BertSum/src/preprocess.py -mode format_to_lines -raw_path 'Tokenized' -save_path 'NextDir'  -log_file 'cnndm2.log' -map_path 'FakeMapPath' -lower

In [0]:
len(os.listdir('NextDir'))

52

In [0]:
os.listdir('NextDir')

['Files.train.9.json',
 'Files.train.8.json',
 '.train.10.json',
 'Files.train.14.json',
 '.test.0.json',
 '.valid.3.json',
 '.train.7.json',
 '.train.17.json',
 '.valid.1.json',
 '.valid.2.json',
 'Files.train.2.json',
 'Files.train.5.json',
 '.train.0.json',
 'Files.valid.1.json',
 '.test.2.json',
 'Files.train.17.json',
 'Files.train.11.json',
 '.train.12.json',
 '.train.3.json',
 'Files.train.3.json',
 '.test.1.json',
 '.train.11.json',
 'Files.train.0.json',
 'Files.test.0.json',
 '.valid.0.json',
 'Files.train.13.json',
 'Files.train.6.json',
 '.valid.4.json',
 'Files.train.1.json',
 '.train.13.json',
 '.train.2.json',
 '.train.9.json',
 '.train.4.json',
 '.train.16.json',
 '.train.1.json',
 'Files.valid.4.json',
 'Files.train.15.json',
 '.train.5.json',
 'Files.train.12.json',
 'Files.test.2.json',
 '.train.8.json',
 '.train.14.json',
 'Files.train.4.json',
 'Files.train.7.json',
 'Files.train.16.json',
 'Files.valid.0.json',
 'Files.valid.3.json',
 'Files.test.1.json',
 'Files.

In [0]:
#It is recommended that you set your save path to your own googl drive, so you don't have to do these steps over again. 

!python BertSum/src/preprocess.py -mode format_to_bert -raw_path 'NextDir' -save_path 'FinalBertData' -oracle_mode greedy -n_cpus 4 -log_file preprocess.log


[('NextDir/Files.train.9.json', Namespace(dataset='', log_file='preprocess.log', lower=True, map_path='../data/', max_nsents=100, max_src_ntokens=200, min_nsents=3, min_src_ntokens=5, mode='format_to_bert', n_cpus=4, oracle_mode='greedy', raw_path='NextDir', save_path='FinalBertData', shard_size=2000), 'FinalBertData/Files.train.9.bert.pt'), ('NextDir/Files.train.8.json', Namespace(dataset='', log_file='preprocess.log', lower=True, map_path='../data/', max_nsents=100, max_src_ntokens=200, min_nsents=3, min_src_ntokens=5, mode='format_to_bert', n_cpus=4, oracle_mode='greedy', raw_path='NextDir', save_path='FinalBertData', shard_size=2000), 'FinalBertData/Files.train.8.bert.pt'), ('NextDir/Files.train.14.json', Namespace(dataset='', log_file='preprocess.log', lower=True, map_path='../data/', max_nsents=100, max_src_ntokens=200, min_nsents=3, min_src_ntokens=5, mode='format_to_bert', n_cpus=4, oracle_mode='greedy', raw_path='NextDir', save_path='FinalBertData', shard_size=2000), 'FinalBer

In [0]:
os.listdir('FinalBertData')

['Files.test.0.bert.pt',
 'Files.train.12.bert.pt',
 'Files.train.5.bert.pt',
 'Files.train.3.bert.pt',
 'Files.train.4.bert.pt',
 'Files.test.1.bert.pt',
 'Files.valid.0.bert.pt',
 'Files.valid.2.bert.pt',
 'Files.test.2.bert.pt',
 'Files.train.16.bert.pt',
 'Files.train.13.bert.pt',
 'Files.train.8.bert.pt',
 'Files.train.11.bert.pt',
 'Files.train.17.bert.pt',
 'Files.train.7.bert.pt',
 'Files.train.6.bert.pt',
 'Files.valid.3.bert.pt',
 'Files.train.0.bert.pt',
 'Files.valid.1.bert.pt',
 'Files.valid.4.bert.pt',
 'Files.train.1.bert.pt',
 'Files.train.9.bert.pt',
 'Files.train.14.bert.pt',
 'Files.train.2.bert.pt',
 'Files.train.15.bert.pt',
 'Files.train.10.bert.pt']

In [0]:
if not os.path.exists('CheckPointFiles'):
    os.makedirs('CheckPointFiles')

In [0]:
#easier to do it from the BertSum directory
%cd BertSum/src

/content/BertSum/src


In [0]:
os.listdir('../../CheckPointFiles')

['events.out.tfevents.1563660545.3bbd5615034d', 'model_step_100.pt']

In [0]:
#First time training
!python train.py -mode train -encoder transformer -dropout 0.2 -bert_data_path '../../FinalBertData/Files' -model_path '../../CheckPointFiles' -lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 10 -save_checkpoint_steps 100 -batch_size 512 -decay_method noam -train_steps 100 -accum_count 2 -log_file bert_transformer -use_interval true -warmup_steps 100 -ff_size 2048 -inter_layers 2 -heads 8

[2019-07-20 22:08:46,757 INFO] Device ID 0
[2019-07-20 22:08:46,757 INFO] Device cuda
[2019-07-20 22:08:46,950 INFO] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz not found in cache, downloading to /tmp/tmphewwegxs
100% 407873900/407873900 [00:05<00:00, 68003497.84B/s]
[2019-07-20 22:08:53,167 INFO] copying /tmp/tmphewwegxs to cache at ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
[2019-07-20 22:08:54,235 INFO] creating metadata file for ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
[2019-07-20 22:08:54,236 INFO] removing temp file /tmp/tmphewwegxs
[2019-07-20 22:08:54,308 INFO] loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca

In [0]:
os.listdir('../../CheckPointFiles')

['events.out.tfevents.1563660545.3bbd5615034d', 'model_step_100.pt']

In [0]:
#Load Checkpoint

# Put the path to your checkpoint in the train_from arg

!python train.py -mode train -encoder transformer -dropout 0.2 -bert_data_path '../../FinalBertData/Files' -model_path '../../CheckPointFiles' -train_from '../../CheckPointFiles/model_step_100.pt' -lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 10 -save_checkpoint_steps 200 -batch_size 512 -decay_method noam -train_steps 200 -accum_count 2 -log_file bert_transformer -use_interval true -warmup_steps 100 -ff_size 2048 -inter_layers 2 -heads 8

[2019-07-20 22:26:21,563 INFO] Device ID 0
[2019-07-20 22:26:21,563 INFO] Device cuda
[2019-07-20 22:26:21,754 INFO] loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
[2019-07-20 22:26:21,754 INFO] extracting archive file ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpxmo7686k
[2019-07-20 22:26:25,808 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

[2019-07-20 22:26:32,523 INFO] Loading checkpoint fro